In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

In [2]:
!pip install -q transformers
!pip install -qU hazm

In [3]:
import math
import numpy as np
import pandas as pd

import hazm

import transformers 
from transformers import AutoTokenizer, AutoConfig
from transformers import TFAutoModelForTokenClassification

import os
from IPython.display import display, HTML, clear_output
from ipywidgets import widgets, Layout

import matplotlib.pyplot as plt

print()
print('tensorflow', tf.__version__)
print('transformers', transformers.__version__)
print('numpy', np.__version__)
print('pandas', pd.__version__)
print()

if tf.test.gpu_device_name() != '/device:GPU:0':
    print()
    print('WARNING: GPU device not found.')
else:
    print()
    print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))


tensorflow 2.3.0
transformers 4.0.1
numpy 1.18.5
pandas 1.1.5




# Setup Neural Based Clasifier

In [4]:
normalizer = hazm.Normalizer()


def cleanize(text):
    """A way to normalize and even clean the text"""
    # clean text
    # do some fns
    return normalizer.normalize(text)

def colorize(text, classes, colors):
    color = sum(classes.reshape((len(colors),1))*colors)
    color = [str(_) for _ in color]
    return f'<span style="background-color: rgb({color[0]}, {color[1]}, {color[2]})">{text}</span>'

def parsbert_ner_load_model(model_name):
    """Load the model"""
    try:
        config = AutoConfig.from_pretrained(model_name)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = TFAutoModelForTokenClassification.from_pretrained(model_name)
        labels = list(config.label2id.keys())

        return model, tokenizer, labels
    except:
        return [None] * 3

def parsbert_ner(texts, utils, visualize=False, mode=None):
    model, tokenizer, labels = utils

    if not model or not tokenizer or not labels:
        return 'Something wrong happened!'
    
    output_predictions = []
    for sequence in texts:
        sequence = cleanize(sequence)
        tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
        inputs = tokenizer.encode(sequence, return_tensors="tf")
        outputs = model(inputs)[0]
        
        if not visualize:
            pass
        else:
            display(HTML("<hr>"))
            O=outputs[:,:,14][0]-4

            B_DAT=outputs[:,:,0][0]
            I_DAT=outputs[:,:,7][0]
            B_TIM=outputs[:,:,6][0]
            I_TIM=outputs[:,:,13][0]

            B_LOC=outputs[:,:,1][0]
            I_LOC=outputs[:,:,8][0]
            B_ORG=outputs[:,:,1][0]
            I_ORG=outputs[:,:,8][0]

            location = B_LOC + B_ORG + I_LOC + I_ORG
            date_and_time = B_TIM + B_DAT + I_TIM + I_DAT

            if mode == "religous times":
                location *= 0
            elif mode == "clock":
                date_and_time *= 0
            else:
                location += 5
                date_and_time += 3

            if visualize > 1:
                index=np.arange(len(tokens))
                width=0.2

                plt.figure(figsize=(15,5))

                plt.bar(index, O, width, color="black", label="None")
                plt.bar(index + width, date_and_time, width, color="blue", label="Date And Time")
                plt.bar(index+2*width, location, width, color="green", label="Location")

                plt.xticks(index+width*1.5, tokens)

                plt.legend(loc="best")
                plt.show()

            
            colorized = []

            for i, token in enumerate(tokens):
                if token not in ["[SEP]", "[CLS]"]:
                    classes = [O[i], location[i], date_and_time[i]]
                    classes = np.exp(classes)/sum(np.exp(classes)) # softmax of classes

                    colors = [(255,255,255), (0, 255, 0), (0, 0, 255)]

                    colorized.append(colorize(token, classes, colors))

            html = " ".join(colorized)
            html = "<p style='direction: rtl; font-size: 20px;'>" + html + "</p>"
            display(HTML(html))

    return output_predictions

# Setup Rule-Based classifier

In [5]:
model_name = 'HooshvareLab/bert-base-parsbert-peymaner-uncased'
print("Loading model...")
utils = parsbert_ner_load_model(model_name)

Loading model...


All model checkpoint layers were used when initializing TFBertForTokenClassification.

All the layers of TFBertForTokenClassification were initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-peymaner-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [6]:
weather = [
           "دمای هوای مسکو در روز جمعه ۹ آبان چند درجه است؟",
           " آب و هوای فردای تهران ساعت 18:35 چه طور است؟",
           "فردا ساعت ۶ بعد از ظهر هوای شیراز چگونه است؟",
           "آب و هوای فردای تهران چطور است؟"
           ]

religous_times = [
                  "اذان ظهر مسکو با اذان مغرب چقدر تفاوت دارد؟",
                  "فاصله ی اذان مغرب تهران و اذان صبح مشهد",
                  "نیمه شب شرعی تورنتو چه زمانی است؟",
                  "اذان ظهر به افق ساری چه وقت است؟",
                  "اذان مغرب فردای قم چه ساعتی است؟",
                  "اذان ظهر تهران چه موقعی است؟ "
                  ]
clock = [
         "اختلاف ساعت تهران و مسکو چند ساعت است؟ ",
         "الان در نیویورک ساعت چند است؟ ",
         "در مونیخ چه زمانی از روز است؟",
         "ساعت چند است؟ "
         ]

date = [
        "تاریخ 05-02-2020 به قمری چه تاریخی است؟",
        "چند روز تا روز جهانی دیابت مانده است؟",
        "دوشنبه هفته ی بعد به تاریخی هجری؟",
        "امروز به میلادی چه روزی است؟",
        "تاریخ شمسی امروز چیست؟",
        "مناسبت 27 رجب چیست؟"
        ]

print("Predicting, please wait...   0-0")
_ = parsbert_ner(weather, utils, visualize=1)
_ = parsbert_ner(clock, utils, visualize=1, mode="clock")


Predicting, please wait...   0-0


# Persian PEYMA NER

In [7]:
#@title Live Playground { display-mode: "form" }
submit_wd = widgets.Button(description='Send', disabled=False, button_style='success', tooltip='Submit')
text_wd = widgets.Textarea(placeholder='Please enter you text ...', rows=5, layout=Layout(width='90%'))
output_wd = widgets.Output()

display(text_wd)
display(submit_wd)
display(output_wd)

def submit_text(sender):
    with output_wd:
        clear_output(wait=True)
        text = text_wd.value

        model_name = 'HooshvareLab/bert-base-parsbert-peymaner-uncased'
        print("Please wait...   0-0")
        output = parsbert_ner([text], utils, visualize=1)


submit_wd.on_click(submit_text)


Textarea(value='', layout=Layout(width='90%'), placeholder='Please enter you text ...', rows=5)

Button(button_style='success', description='Send', style=ButtonStyle(), tooltip='Submit')

Output()